### Chat History OakLang Graph
Here we'll be interacting with a server that's exposing a chat bot with memmory history being persisted on the backend.

In [1]:
import uuid
from langserve import RemoteRunnable

session_id = str(uuid.uuid4())
cheese = RemoteRunnable("http://localhost:8000/cheese")

Let's create a prompt composed of a system message and a human message.

In [2]:
# define the thread_id
config = {"configurable": {"thread_id": "001"}}

In [3]:
# Define the input and invoke
user_input = "The pokemon I saw fishing was white? What is the evolution of Houndstone?"
response = cheese.invoke({"messages": [("human", user_input)]}, config)
response


{'messages': [['human',
   'The pokemon I saw fishing was white? What is the evolution of Houndstone?'],
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-10T15:35:54.4260337Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'EvolutionTool', 'arguments': {'pokemon_name': 'Houndstone'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 4352732200, 'load_duration': 2464299900, 'prompt_eval_count': 408, 'prompt_eval_duration': 644000000, 'eval_count': 21, 'eval_duration': 631000000}, id='run-693b2e4f-840a-4e3d-b670-7f5534660727-0', tool_calls=[{'name': 'EvolutionTool', 'args': {'pokemon_name': 'Houndstone'}, 'id': '2382476a-561a-4112-979b-394626152053', 'type': 'tool_call'}], usage_metadata={'input_tokens': 408, 'output_tokens': 21, 'total_tokens': 429}),
  ToolMessage(content='["greavard", "houndstone"]', name='EvolutionTool', tool_call_id='2382476a-561a-4112-979b-394626152053'),
  A

In [ ]:
# to filter the message
response["messages"][-1].content

In [ ]:
# continuing chatting in the same thread
user_input = "The other day I saw a rat in the streets? what could be? could you give me random movement wicht I can show?"
cheese.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

In [ ]:
# check the memory
user_input = "oh thats sounds nice, could you give me more randoms movements for that pokemon? and what is its evolution?"
cheese.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

In [ ]:
# check the memory with new config
config = {"configurable": {"thread_id": "002"}}
user_input = "My name is Angel, hello"
cheese.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

In [ ]:
# check the memory
user_input = "What was my name?"
cheese.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

### Live Chat OakLang Graph

In [ ]:
## Follow the nodes via stream

# define the thread_id
config = {"configurable": {"thread_id": "004"}}

# define the input
user_input = "Do you know what is an atom?"

for output in cheese.stream({"messages": [("human", user_input)]}, config):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

In [ ]:
## Live chat via stream

# define the thread_id
config = {"configurable": {"thread_id": "007"}}

while True:
    user_input = input("Human: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Assistant: Goodbye!")
        break
    else:
        print(f"Human: {user_input}") # You add this to see the user input
    for event in cheese.stream({"messages": [("human", user_input)]}, config): 
        for key, value in event.items():
            if key == 'agent' and len(value["messages"][-1].content) > 0:
                print("Assistant:", value["messages"][-1].content)
                

### Request API

In [ ]:
import requests

response = requests.post(
    url='http://localhost:8000/oak/invoke',
    json={
        'input': {
            "messages": [("human", "Hello my name is BOB")],
        },
        'config': {
            'configurable': {
                "thread_id": "003"
            }
        }
    }
)

print(response.json()["output"]["messages"][-1])

In [ ]:
import requests

response = requests.post(
    url='http://localhost:8000/oak/invoke',
    json={
        'input': {
            "messages": [("human", "What was my name?")],
        },
        'config': {
            'configurable': {
                "thread_id": "003"
            }
        }
    }
)

print(response.json()["output"]["messages"][-1])